In [2]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [48]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out.basics import create_dir, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.in_out import PointCloudDataSet, load_point_clouds_from_filenames
from tf_lab.point_clouds.encoders_decoders import encoder_with_convs_and_symmetry, decoder_with_fc_only,\
encoder_with_convs_and_symmetry_and_fc

from tf_lab.in_out.basics import read_saved_epochs
from tf_lab.data_sets.articulated_human_poses import pc_loader



In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
n_pc_points = 2048
do_training = True
first_time_running = True
load_pretrained_model = False
load_epoch = None
seed = 42

In [44]:
max_training_epochs = 1000
loss = 'chamfer'
bneck_size = 16

experiment_name = '_'.join(['human_articulation_all_data_no_max', str(n_pc_points), 
                           'pts_bneck', str(bneck_size), loss
                           ])

top_data_dir = '/orions4-zfs/projects/optas/DATA/'
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/', experiment_name)
create_dir(train_dir)

'/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/human_articulation_all_data_no_max_2048_pts_bneck_16_chamfer'

In [13]:
pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Articulated_Mesh_Animations', str(n_pc_points))
all_data_files = [f for f in files_in_subdirs(pclouds_path, '.ply')]
pclouds, model_names, _ = load_point_clouds_from_filenames(all_data_files, n_threads=20, loader=pc_loader, verbose=True)
train_data = PointCloudDataSet(pclouds, labels=model_names)

In [62]:
b_norm = False
encoder_args = {'args_of_patrial':
                {'n_filters': [64, 128, 256, bneck_size],
                'filter_sizes': [20, 10, 10, 5],
                'strides': [1, 1, 2, 2],
                'symmetry': None,
                'b_norm': b_norm
                }}
n_input = [n_pc_points, 3]

decoder_args = {'layer_sizes': [1024, 2048, np.prod(n_input)],
               'b_norm': b_norm}

In [63]:
from functools import partial
encoder = partial(encoder_with_convs_and_symmetry_and_fc, fc_nout=bneck_size)
encoder.__name__ = 'partial_fc_encoder_with_bneck_size'

In [64]:
if load_pretrained_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
    if conf.train_dir != train_dir:
        conf.train_dir = train_dir
    conf.save(osp.join(train_dir, 'configuration'))
else:
    conf = Conf(
                n_input = [n_pc_points, 3],
                loss = loss,
                training_epochs = 200,
                batch_size = 50,
                denoising = False,
                learning_rate = 0.0005,
                train_dir = train_dir,              
                loss_display_step = 1,
                saver_step = 20,
                z_rotate = False,
                encoder = encoder,
                decoder = decoder_with_fc_only,
                encoder_args = encoder_args,
                decoder_args = decoder_args
               )    
    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))

In [65]:
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

In [66]:
if load_pretrained_model:
    if load_epoch is None: # load last saved.
        saved_epochs = read_saved_epochs(conf.train_dir)
        load_epoch = saved_epochs[-1]        
    ae.restore_model(conf.train_dir, load_epoch, verbose=True)

In [67]:
if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)    
    train_stats = ae.train(train_data, conf, log_file=fout)
    fout.close()

('Epoch:', '0001', 'training time (minutes)=', '0.0332', 'loss=', '0.007815272')
INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/human_articulation_all_data_no_max_2048_pts_bneck_16_chamfer/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0002', 'training time (minutes)=', '0.0302', 'loss=', '0.003526474')
('Epoch:', '0003', 'training time (minutes)=', '0.0313', 'loss=', '0.003263139')
('Epoch:', '0004', 'training time (minutes)=', '0.0306', 'loss=', '0.002832258')
('Epoch:', '0005', 'training time (minutes)=', '0.0304', 'loss=', '0.002483313')
('Epoch:', '0006', 'training time (minutes)=', '0.0303', 'loss=', '0.002144296')
('Epoch:', '0007', 'training time (minutes)=', '0.0312', 'loss=', '0.001900172')
('Epoch:', '0008', 'training time (minutes)=', '0.0311', 'loss=', '0.001734968')
('Epoch:', '0009', 'training time (minutes)=', '0.0317', 'loss=', '0.001602206')
('Epoch:', '0010', 'training time (minutes)=', '0.0308', 'loss=', '0.0